In [36]:
import json
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import collections
collections.Callable = collections.abc.Callable
import pymysql
import pandas as pd
import datetime


# json file로 변환하는 함수
def transform_to_json(dic, file_name):
    with open(file_name, 'w', encoding = 'UTF8') as f:
        json.dump(dic, f, ensure_ascii=False, indent = '\t')
        
        
# text 데이터 정제 함수        
def clean_data(string):
    regexed_str = re.sub("[\n]+", " ", string)
    regexed_str = re.sub("[\t]+", " ", string)
    return_string = regexed_str.replace("\n", " ")
    return return_string.strip()


# 테이블 형태 데이터 처리 함수
def small_table_parser(contents_node, header_node):
    total_lst = []
    for contents_idx in range(0, len(contents_node), len(header_node.find_all("th"))):
        sub_dict = {}
        for th in range(len(header_node.find_all("th"))):
            sub_dict[header_node.find_all("th")[th].text] = clean_data(contents_node[contents_idx + th].text)
        total_lst.append(sub_dict)
    return total_lst


# 표준 데이터 크롤러
def std_crawler(whole_dict, lst_page_soup, driver, std_idx):
    page_list_data = lst_page_soup.find_all("tbody")[1]
    row_lst = page_list_data.find_all("tr")
    
    driver.implicitly_wait(100)
    
    for standard in range(std_idx, len(row_lst) + 1):
        
        # data 개수 체크
        num = int(lst_page_soup.select("#tabs-container > div.table.list > table > tbody > tr:nth-of-type({}) > td:nth-of-type(1)".format(standard))[0].text)
        
        driver.implicitly_wait(100)
        
        #crawling 작업
        try:
            driver.find_element(By.XPATH, '//*[@id="tabs-container"]/div[2]/table/tbody/tr[{}]/td[2]/a'.format(standard)).click()
            
            std = {}
            
            result_html = driver.page_source
            soup = BeautifulSoup(result_html, 'html.parser')
            
            # 테이블 row 개수
            table_data = soup.find("div", {'class':'table view'})
            table_data = table_data.find('tbody').children

            table_lst = []
            for child in table_data:
                table_lst.append(child)
            for i in table_lst:
                if i == "\n":
                    table_lst.remove(i)
            
                                    
            for row in range(1, len(table_lst) + 1):
                selector = "#contents > div > div.content_inner > div.table.view > table > tbody > "
                title = soup.select(selector + "tr:nth-of-type({}) > th".format(row))[0].text
                
                # 인용표준, 기술기준 데이터 / index text를 비교하는 if 문으로 바꾸기 / 가독성 고려
                if title == "인용표준" or title == "기술기준":
                    title = soup.select(selector + "tr:nth-of-type({}) > th".format(row))[0].text
                    content = soup.select(selector + "tr:nth-of-type({}) > td".format(row))[0].text
                    
                    title_2 = soup.select(selector + "tr:nth-of-type({}) > th:nth-of-type(2)".format(row))[0].text
                    content_2 = soup.select(selector + "tr:nth-of-type({}) > td:nth-of-type(2)".format(row))[0].text
                    
                    
                    content = clean_data(content)
                    content_2 = clean_data(content_2)
                    
                    std[title] = content
                    std[title_2] = content_2
                    
                #테이블 데이터 처리 / 유형별로 파악해서 if 문 바꾸기 / 아예 클래스를 만들어서 기존 항목에 추가 제거의 경우에 처리할 경우 / 테이블 파싱하는 함수 만들기 (colspan 예외 처리도)
                elif title == "국제표준 부합화" or title == "표준이력사항" or title == "인증심사기준":
                    std[title] = []
                    
                    # 국제표준 부합화
                    if title == "국제표준 부합화":
                        sub_dict = {}
                        data = soup.find("div", {'class':"table list gray"})
                        contents = data.find_all("td")
                        
                        if "colspan" in contents[0].attrs:
                            for j in range(int(contents[0].attrs["colspan"])):
                                sub_dict[data.find_all("th")[j].text] = contents[0].text
                            std[title].append(sub_dict)
                            contents = contents[1:]
                            
                        table_data_lst = small_table_parser(contents, data)
                        for tb_data in table_data_lst:
                            std[title].append(tb_data)
                        
                    #표준이력사항
                    elif title == "표준이력사항":
                        data = soup.find_all("div", {'class':"table list gray"})[1]
                        contents = data.find_all("td")
                        
                        table_data_lst = small_table_parser(contents, data)
                        std[title] = table_data_lst
                            
                    #인증심사기준
                    else:
                        sub_dict = {}
                        data = soup.find_all("div", {'class':"table list gray"})[2]
                        contents = data.find_all("td")
                        if len(contents) == 1:
                            for i in range(int(contents[0].attrs["colspan"])):
                                sub_dict[data.find_all("th")[i].text] = contents[0].text
                            std[title].append(sub_dict)
                        
                        else:
                            table_data_lst = small_table_parser(contents, data)
                            std[title] = table_data_lst

                # 나머지 데이터
                else:
                    content = soup.select(selector + "tr:nth-of-type({}) > td".format(row))[0].text
                    content = clean_data(content)
                    std[title] = content
                    
            now = time.localtime()
            now = "%04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
            std['crawled_time'] = now
            whole_dict['stds'].append(std)
            
            driver.back()
            driver.refresh()
            
        except:
            print("Crawling error, No : ", num)
            is_success = False
            return whole_dict, is_success, standard
    
        driver.implicitly_wait(100)
    
    std_idx = 1    
    is_success = True
    return whole_dict, is_success, std_idx


# 페이지 넘기는 크롤러
def page_crawler(start_page, chrome_path, url, std_idx):
    driver = webdriver.Chrome(executable_path=chrome_path)
    driver.get(url)
    
    driver.find_element(By.CLASS_NAME, "last").click()
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    last_page = soup.find_all("a", {"class": "on"})[1].text
    driver.find_element(By.CLASS_NAME, "first").click()
    
    std_dict = {}
    std_dict['stds'] = []
    
    start_page = int(start_page)
    if start_page % 10 == 0:
        num_next = start_page // 10 - 1
    else:
        num_next = start_page // 10

    for i in range(num_next):
        driver.find_element(By.CLASS_NAME, 'next').click()
        
    page_source = driver.page_source
    lst_page_soup = BeautifulSoup(page_source, 'html.parser')

    # 페이지 버튼 갯수
    page_button_data = lst_page_soup.find("div", {'class':"page"})
    li_lst = page_button_data.find_all("li")
    li_lst = [button.text for button in li_lst]
    start = li_lst.index(str(start_page))
    
    is_success = False
    page_num = 0
    
    try:
        while True:
            for page in range(start + 1, len(li_lst) - 1):
                driver.find_element(By.XPATH, '//*[@id="tabs-container"]/div[3]/div/div/ul/li[{}]/a'.format(page)).click()
                page_num = driver.find_element(By.XPATH, '//*[@id="tabs-container"]/div[3]/div/div/ul/li[{}]/a'.format(page)).text
                page_source = driver.page_source
                lst_page_soup = BeautifulSoup(page_source, 'html.parser')
                driver.implicitly_wait(100)
                
                std_dict, is_success, std_idx = std_crawler(std_dict, lst_page_soup, driver, std_idx)
                
                
                if page_num == last_page:
                    is_success = True
                    return std_dict, int(page_num), is_success, std_idx
                
                elif is_success == False:
                    return std_dict, int(page_num), is_success, std_idx
                    
                
            # 다시 1페이지부터
            start = 2
            
            driver.find_element(By.CLASS_NAME, 'next').click()
            
    except:
        return std_dict, int(page_num), is_success, std_idx
    

# json 파일 dataframe으로 읽는 함수
def json_reader(filename):
    with open (filename, "r", encoding = 'UTF8') as f:
        data = json.load(f)
    
    df = pd.json_normalize(data['stds'])
    return df


# DB에 테이블이 생성되어 있다면 테이블에 데이터 넣는 함수
def db_process(filename, host, password, schema_name, table_name):
    try:
        db = pymysql.connect(host = host, port = 3306, user = 'root', password = password,
                        db = schema_name, charset = 'utf8')   # charset: 인코딩 설정
    except: 
        print("Can not connent to DB server. Please check the connection")
        return
    
    cursor = db.cursor()

    sql = 'INSERT INTO ' + table_name + ' (doc_num, doc_name_ko, doc_name_en, publish_date, final_date, json_data, crawled_time, tag) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'

    output_df = json_reader(filename)
    output_df = output_df.reset_index()

    with open (filename, "r", encoding = 'UTF8') as f:
        data = json.load(f)
        
    lst = data['stds']
            
    for i in range(len(output_df)):
        json_data = json.dumps(lst[i])
        try:
            cursor.execute(sql, (output_df.loc[i]["표준번호"], output_df.loc[i]["표준명(한글)"], output_df.loc[i]["표준명(영문)"], output_df.loc[i]["제정일"], output_df.loc[i]["최종개정확인일"], json_data, output_df.loc[i]["crawled_time"], filename[7:15]))
        except:
            if len(output_df.loc[i]["제정일"]) == 0:
                cursor.execute(sql, (output_df.loc[i]["표준번호"], output_df.loc[i]["표준명(한글)"], output_df.loc[i]["표준명(영문)"], None, output_df.loc[i]["최종개정확인일"], json_data, output_df.loc[i]["crawled_time"], filename[7:15]))
            else:
                print("There is a problem while inserting data number : ", i)
    db.commit()

    db.close()
    print("Done with inserting data to DB")
    return


# main 함수 = crawling --> json data --> DB 연동 및 데이터 inserting
def main(last_page_no, max_retry_num, chrome_path, url, host, password, schema_name, table_name):
    data_list = {}
    data_list['stds'] = []
    current_retry_count = 0
    std_idx = 1
    now = datetime.datetime.now()
    output_name = "output_{}.json".format(now.strftime('%Y%m%d'))
    
    while True:
        if current_retry_count >= max_retry_num:
            print("number of retry exceeds maximum retry number")
            print("last page of crawling is ", last_page_no)
            transform_to_json(data_list, output_name)
            break
            # return data_list, is_success, current_retry_count
        
        current_data_list, end_page_no, is_success, std_idx = page_crawler(last_page_no, chrome_path, url, std_idx)
        
        # merge current_data_list to data_list 
        for i in current_data_list['stds']:
            if i not in data_list['stds']:
                data_list['stds'].append(i)
                
        # if it couldn't reach the last page
        if is_success == False:
            current_retry_count = current_retry_count + 1
            transform_to_json(data_list, output_name)
            last_page_no = end_page_no
            
            print("Trying to crawl the error page : ", last_page_no)
            
            driver_2 = webdriver.Chrome(executable_path=chrome_path)
            driver_2.get(url)

            for i in range(end_page_no // 10):
                driver_2.find_element(By.CLASS_NAME, 'next').click()
                
            page_source = driver_2.page_source
            lst_page_soup = BeautifulSoup(page_source, 'html.parser')
            
            sub_dict = {}
            sub_dict['stds'] = []

            not_success_dict, is_sucess, standard = std_crawler(sub_dict, lst_page_soup, driver_2, std_idx)
            
            for i in not_success_dict['stds']:
                data_list['stds'].append(i)
            
            print("Crawling is finished with the error page : ", last_page_no)

            last_page_no = last_page_no + 1

            print("Recrawling from the page number : ", last_page_no)
        
        # if it reaches the last page
        elif is_success:
            print("Done with crawling")
            transform_to_json(data_list, output_name)
            break
            # return data_list, is_success, current_retry_count
            
    return db_process(output_name, host, password, schema_name, table_name)

In [39]:
url = "https://standard.go.kr/KSCI/standardIntro/getStandardSearchList.do?menuId=919&topMenuId=502"
main(890, 10, r"C:\Users\gihok\chatbot\chromedriver.exe", url, "192.168.0.124", "linuxer", "std_crawled_data", "std_data_check")

C:\Users\gihok\AppData\Local\Temp\ipykernel_11200\3232336731.py:163: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_path)


Crawling error, No :  8901
Trying to crawl the error page :  891


C:\Users\gihok\AppData\Local\Temp\ipykernel_11200\3232336731.py:303: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver_2 = webdriver.Chrome(executable_path=chrome_path)


Crawling is finished with the error page :  891
Recrawling from the page number :  892
Crawling error, No :  10101
Trying to crawl the error page :  1011
Crawling is finished with the error page :  1011
Recrawling from the page number :  1012
Crawling error, No :  10201
Trying to crawl the error page :  1021
Crawling is finished with the error page :  1021
Recrawling from the page number :  1022


AttributeError: 'NoneType' object has no attribute 'click'

In [18]:
json_reader("output_20220829.json")

,표준번호,표준명(한글),표준명(영문),KS원문보기,표준분야,표준구분,제정일,최종개정확인일,기술심의회,전문위원회,...,담당자명,담당자 연락처,국제표준 부합화,인용표준,기술기준,ICS Code,표준이력사항,인증심사기준,crawled_time,대체 표준
0,KS C IEC60601-2-45,의료용 전기기기 ─ 제2-45부: 유방 촬영 X선 장치 및 유방 촬영 입체 정위 기...,Medical electrical equipment- Part 2-45: Parti...,표준 원문보기,(P) 의료 - (P12) 전자의료기기,제품,2005-06-30,2017-09-29,"의료용전기제품 기술심의회(A,C,P,X)",IEC/TC 62/SC B(진단영상장치),...,정상오,043-719-5660,"[{'대응국제표준': 'IEC 60601-2-45 Ed. 3.1', '부합화수준':...",0 건,0 건,11.040.50 (방사선 및 기타 진단 기기),"[{'변경일자': '2005-06-30', '구분': '제정', '고시번호': '2...","[{'제/개정일자': '관련 자료가 없습니다.', '구분': '관련 자료가 없습니다...",2022/08/29 09:50:13,NaN
1,KS C IEC60601-2-46,의료용 전기기기 ─ 제2-46부: 수술대의 기본안전 및 필수성능에 관한 개별 요구사항,Medical electrical equipment ─ Part 2-46: Part...,표준 원문보기,(C) 전기전자 - (C22) 기타,전달,2005-06-30,2021-12-28,"의료용전기제품 기술심의회(A,C,P,X)",IEC/TC 62/SC D(의료용전자기기),...,정상오,043-719-5660,"[{'대응국제표준': 'IEC 60601-2-46 Ed. 2.0', '부합화수준':...",5 건,0 건,11.140 (병원 장비),"[{'변경일자': '2005-06-30', '구분': '제정', '고시번호': '2...","[{'제/개정일자': '관련 자료가 없습니다.', '구분': '관련 자료가 없습니다...",2022/08/29 09:50:30,NaN
2,KS C IEC60601-2-47,의료용 전기기기 ─ 제2-47부: 휴대용 심전도 장치의 기본 안전 및 필수 성능에 ...,Medical eoectrical equipment ─ Part 2-47: Part...,표준 원문보기,(C) 전기전자 - (C22) 기타,전달,2005-06-30,2017-09-29,"의료용전기제품 기술심의회(A,C,P,X)",IEC/TC 62/SC D(의료용전자기기),...,정상오,043-719-5660,"[{'대응국제표준': 'IEC 60601-2-47 Ed. 2.0', '부합화수준':...",2 건,0 건,11.040.55 (),"[{'변경일자': '2005-06-30', '구분': '제정', '고시번호': '2...","[{'제/개정일자': '관련 자료가 없습니다.', '구분': '관련 자료가 없습니다...",2022/08/29 09:50:35,NaN
3,KS C IEC60601-2-49 (폐지),의료용 전기기기 ─ 제2-49부: 다기능 환자 감시기의 기본 안전 및 필수 성능에...,Medical electrical equipment - Part 2-49 : Par...,NaN,(P) 의료 - (P12) 전자의료기기,전달,2005-06-30,2020-12-30,"의료용전기제품 기술심의회(A,C,P,X)",IEC/TC 62/SC D(의료용전자기기),...,정상오,043-719-5660,"[{'대응국제표준': 'IEC 60601-2-49 Ed. 2.0', '부합화수준':...",0 건,0 건,11.040.55 (),"[{'변경일자': '2005-06-30', '구분': '제정', '고시번호': '2...","[{'제/개정일자': '관련 자료가 없습니다.', '구분': '관련 자료가 없습니다...",2022/08/29 09:50:40,
4,KS C IEC60601-2-5,의료용 전기기기 ─ 제2-5부: 초음파 물리치료기의 기본안전과 필수성능에 관한 개별...,Medical electrical equipment ─ Part2-5: Partic...,표준 원문보기,(C) 전기전자 - (C22) 기타,전달,,2021-12-08,"의료용전기제품 기술심의회(A,C,P,X)",IEC/TC 62/SC D(의료용전자기기),...,정상오,043-719-5660,"[{'대응국제표준': 'IEC 60601-2-5 Ed. 3.0', '부합화수준': ...",0 건,0 건,11.040.60 (치료장비),"[{'변경일자': '2001-12-28', '구분': '제정', '고시번호': '2...","[{'제/개정일자': '관련 자료가 없습니다.', '구분': '관련 자료가 없습니다...",2022/08/29 09:50:45,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,KS C IEC60642-3,압전 세라믹 공진자 — 제3부: 표준 개요,Piezoelectric ceramic resonators - Part 3: Sta...,표준 원문보기,(C) 전기전자 - (C18) 전기·전자·통신 부품,전달,2002-12-30,2022-08-25,전자 기술심의회(C),"압전, 유전 소자",...,서순규,043-870-5362,"[{'대응국제표준': 'IEC 60642-3 Ed. 1.0', '부합화수준': 'I...",0 건,0 건,31.140 (압전 장치),"[{'변경일자': '2002-12-30', '구분': '제정', '고시번호': '2...","[{'제/개정일자': '관련 자료가 없습니다.', '구분': '관련 자료가 없습니다...",2022/08/29 10:03:14,NaN
76,KS C IEC60644 (폐지),모터회로용 고압 퓨즈링크의 일반 요구 사항 (폐지),SPECIFICATION FOR HIGH-VOLTAGE FUSE-LINK FOR M...,NaN,(C) 전기전자 - (C20) 배선·전기 기기,제품,2003-12-29,2014-01-31,전기응용 기술심의회(C),퓨즈류,...,서순규,043-870-5362,"[{'대응국제표준': '관련 자료가 없습니다.', '부합화수준': '관련 자료가 없...",0 건,0 건,29.120.50 (휴즈 및 기타 과전류 방지 장치),"[{'변경일자': '2003-12-29', '구분': '제정', '고시번호': '2...","[{'제/개정일자': '관련 자료가 없습니다.', '구분': '관련 자료가 없습니다...",2022/08/29 10:03:18,
77,KS C IEC60645-1 (폐지),청력계-제1부:순음 청력계 (폐지),Audiometers - Part 1 : Pure-tone audiometers,NaN,(C) 전기전자 - (C13) 측정·시험용 기계기구,제품,2002-10-11,2014-01-31,전기응용 기술심의회(C),전기음향,...,김성희,043-870-5365,"[{'대응국제표준': 'IEC 60645-1', '부합화수준': 'IDT 일치'},...",0 건,0 건,13.140 (인간 관련 소음) 17.140.01 (...,"[{'변경일자': '2002-10-11', '구분': '제정', '고시번호': '2...","[{'제/개정일자': '관련 자료가 없습니다.', '구분': '관련 자료가 없습니다...",2022/08/29 10:03:23,
78,KS C IEC60645-2 (폐지),청력계-제2부:언어음 청각 검사에 사용하는 기기 (폐지),Audiometers-Part2: Equipment for speech audiom...,NaN,(C) 전기전자 - (C13) 측정·시험용 기계기구,제품,2002-10-11,2014-01-31,전기응용 기술심의회(C),전기음향,...,김성희,043-870-5365,"[{'대응국제표준': 'IEC 60645-2', '부합화수준': 'IDT 일치'},...",0 건,0 건,17.140.50 (전자 음향),"[{'변경일자': '2002-10-11', '구분': '제정', '고

In [22]:
driver = webdriver.Chrome(executable_path=r"C:\Users\gihok\chatbot\chromedriver.exe")
driver.get("https://standard.go.kr/KSCI/standardIntro/getStandardSearchList.do?menuId=919&topMenuId=502")

for i in range(101):
    driver.find_element(By.CLASS_NAME, 'next').click()

C:\Users\gihok\AppData\Local\Temp\ipykernel_11200\1244300169.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\gihok\chatbot\chromedriver.exe")


In [23]:
page_source = driver.page_source
lst_page_soup = BeautifulSoup(page_source, 'html.parser')

In [24]:
driver.find_element(By.XPATH, '//*[@id="tabs-container"]/div[2]/table/tbody/tr[{}]/td[3]/a'.format(1)).click()

In [25]:
whole_dict = {}
whole_dict['stds'] = []

In [26]:
sample_dict, is_sucess, standard = std_crawler(whole_dict, lst_page_soup, driver, 1)

In [27]:
for i in sample_dict['stds']:
    print(i)
    break

{'표준번호': 'KS C IEC61249-2-18', '표준명(한글)': '인쇄회로기판 및 기타 전자배선용 재료 — 제2-18부: 동박적층 난연등급(수직연소시험)의 유리섬유 비직조 폴리에스테르 동박적층판 및 무동박적층판', '표준명(영문)': 'Materials for printed boards and other interconnecting structures — Part 2-18: Reinforced base materials, clad and unclad — Polyester non-woven fibreglass reinforced laminated sheet of defined flammability (vertical burning test), copper-clad', 'KS원문보기': '표준 원문보기', '표준분야': '(C) 전기전자 - (C22) 기타', '표준구분': '방법', '제정일': '2011-07-28', '최종개정확인일': '2021-08-27', '기술심의회': '전자 기술심의회(C)', '전문위원회': '전자실장기술', '적용범위': '', '표준개발협력기관': '한국실장산업협회', '담당부처': '국가기술표준원', '담당부서': '전기전자정보표준과', '담당자명': '조형민', '담당자 연락처': '043-870-5364', '국제표준 부합화': [{'대응국제표준': '관련 자료가 없습니다.', '부합화수준': '관련 자료가 없습니다.'}], '인용표준': '5 건', '기술기준': '0 건', 'ICS Code': '31.180 (회로기판)', '표준이력사항': [{'변경일자': '2011-07-28', '구분': '제정', '고시번호': '2011-0255', '제정,개정,폐지 사유': 'PCB 기판의 소재인 동박적층판의 기초원자재의 원천기술 확보를 도모하고 향후 차세대 기판 및 소재에 대한 글로벌 경쟁력 확보를 강화하고자 함.', '신구대비표': ''}, {'변경일자': '2016-12-02', '구분': '확인', '고시번

In [4]:
json_reader('output_20220829.json')

,표준번호,표준명(한글),표준명(영문),KS원문보기,표준분야,표준구분,제정일,최종개정확인일,기술심의회,전문위원회,...,담당자명,담당자 연락처,국제표준 부합화,인용표준,기술기준,ICS Code,표준이력사항,인증심사기준,crawled_time,대체 표준
0,KS C IEC60601-2-45,의료용 전기기기 ─ 제2-45부: 유방 촬영 X선 장치 및 유방 촬영 입체 정위 기...,Medical electrical equipment- Part 2-45: Parti...,표준 원문보기,(P) 의료 - (P12) 전자의료기기,제품,2005-06-30,2017-09-29,"의료용전기제품 기술심의회(A,C,P,X)",IEC/TC 62/SC B(진단영상장치),...,정상오,043-719-5660,"[{'대응국제표준': 'IEC 60601-2-45 Ed. 3.1', '부합화수준':...",0 건,0 건,11.040.50 (방사선 및 기타 진단 기기),"[{'변경일자': '2005-06-30', '구분': '제정', '고시번호': '2...","[{'제/개정일자': '관련 자료가 없습니다.', '구분': '관련 자료가 없습니다...",2022/08/29 09:10:06,NaN
1,KS C IEC60601-2-46,의료용 전기기기 ─ 제2-46부: 수술대의 기본안전 및 필수성능에 관한 개별 요구사항,Medical electrical equipment ─ Part 2-46: Part...,표준 원문보기,(C) 전기전자 - (C22) 기타,전달,2005-06-30,2021-12-28,"의료용전기제품 기술심의회(A,C,P,X)",IEC/TC 62/SC D(의료용전자기기),...,정상오,043-719-5660,"[{'대응국제표준': 'IEC 60601-2-46 Ed. 2.0', '부합화수준':...",5 건,0 건,11.140 (병원 장비),"[{'변경일자': '2005-06-30', '구분': '제정', '고시번호': '2...","[{'제/개정일자': '관련 자료가 없습니다.', '구분': '관련 자료가 없습니다...",2022/08/29 09:10:11,NaN
2,KS C IEC60601-2-47,의료용 전기기기 ─ 제2-47부: 휴대용 심전도 장치의 기본 안전 및 필수 성능에 ...,Medical eoectrical equipment ─ Part 2-47: Part...,표준 원문보기,(C) 전기전자 - (C22) 기타,전달,2005-06-30,2017-09-29,"의료용전기제품 기술심의회(A,C,P,X)",IEC/TC 62/SC D(의료용전자기기),...,정상오,043-719-5660,"[{'대응국제표준': 'IEC 60601-2-47 Ed. 2.0', '부합화수준':...",2 건,0 건,11.040.55 (),"[{'변경일자': '2005-06-30', '구분': '제정', '고시번호': '2...","[{'제/개정일자': '관련 자료가 없습니다.', '구분': '관련 자료가 없습니다...",2022/08/29 09:10:16,NaN
3,KS C IEC60601-2-49 (폐지),의료용 전기기기 ─ 제2-49부: 다기능 환자 감시기의 기본 안전 및 필수 성능에...,Medical electrical equipment - Part 2-49 : Par...,NaN,(P) 의료 - (P12) 전자의료기기,전달,2005-06-30,2020-12-30,"의료용전기제품 기술심의회(A,C,P,X)",IEC/TC 62/SC D(의료용전자기기),...,정상오,043-719-5660,"[{'대응국제표준': 'IEC 60601-2-49 Ed. 2.0', '부합화수준':...",0 건,0 건,11.040.55 (),"[{'변경일자': '2005-06-30', '구분': '제정', '고시번호': '2...","[{'제/개정일자': '관련 자료가 없습니다.', '구분': '관련 자료가 없습니다...",2022/08/29 09:10:21,
4,KS C IEC60601-2-5,의료용 전기기기 ─ 제2-5부: 초음파 물리치료기의 기본안전과 필수성능에 관한 개별...,Medical electrical equipment ─ Part2-5: Partic...,표준 원문보기,(C) 전기전자 - (C22) 기타,전달,,2021-12-08,"의료용전기제품 기술심의회(A,C,P,X)",IEC/TC 62/SC D(의료용전자기기),...,정상오,043-719-5660,"[{'대응국제표준': 'IEC 60601-2-5 Ed. 3.0', '부합화수준': ...",0 건,0 건,11.040.60 (치료장비),"[{'변경일자': '2001-12-28', '구분': '제정', '고시번호': '2...","[{'제/개정일자': '관련 자료가 없습니다.', '구분': '관련 자료가 없습니다...",2022/08/29 09:10:26,NaN
5,KS C IEC60601-2-50,의료용 전기기기 ─ 제2-50부: 신생아황달치료용광선조사기의 기본 안전 및 필수 성...,Medical electrical equipment ─ Part 2-50: Part...,표준 원문보기,(P) 의료 - (P12) 전자의료기기,제품,,2017-12-29,"의료용전기제품 기술심의회(A,C,P,X)",IEC/TC 62/SC D(의료용전자기기),...,정상오,043-719-5660,"[{'대응국제표준': 'IEC 60601-2-50 Ed. 2.1', '부합화수준':...",2 건,0 건,11.040.60 (치료장비),"[{'변경일자': '2002-03-25', '구분': '제정', '고시번호': '2...","[{'제/개정일자': '관련 자료가 없습니다.', '구분': '관련 자료가 없습니다...",2022/08/29 09:10:31,NaN
6,KS C IEC60601-2-51 (폐지),의료용 전기기기 - 제2-51부: 단일 및 다중 채널 심전계 기록 및 분석 안전성 ...,Medical electrical equipment - Part 2-51: Part...,NaN,(C) 전기전자 - (C22) 기타,전달,2008-08-08,2013-12-31,전력기기 기술심의회(C),,...,서순규,043-870-5362,"[{'대응국제표준': '관련 자료가 없습니다.', '부합화수준': '관련 자료가 없...",0 건,0 건,01.040.11 (의료공학 (용어)) 11.040 ...,"[{'변경일자': '2008-08-08', '구분': '제정', '고시번호': '2...","[{'제/개정일자': '관련 자료가 없습니다.', '구분': '관련 자료가 없습니다...",2022/08/29 09:10:36,
7,KS C IEC60601-2-54,의료용 전기기기 ─ 제2-54부: 촬영 및 투시용 X선 장치의 기본 안전과 필수 성...,Medical electrical equipments ─ Parts 2-54: Pa...,표준 원문보기,(P) 의료 - (P12) 전자의료기기,전달,2012-01-02,2020-12-30,"의료용전기제품 기술심의회(A,C,P,X)",IEC/TC 62/SC B(진단영상장치),...,정상오,043-719-5660,"[{'대응국제표준': 'IEC 60601-2-54 Ed. 1.2', '부합화수준':...",8 건,0 건,11.040.50 (방사선 및 기타 진단 기기),"[{'변경일자': '2012-01-02', '구분': '제정', '고시번호': '2...","[{'제/개정일자': '관련 자료가 없습니다.', '구분': '관련 자료가 없습니다...",2022/08/29 09:10:41,NaN
8,KS C IEC60601-2-6,의료용 전기기기 ─ 제2-6부: 극초단파자극기의 기본 안전 및 필수 성능에 관한 개...,Medical electrical equipment — Part 2-6: Parti...,표준 원문보기,(C) 전기전자 - (C22) 기타,전달,2005-06-30,2021-06-28,"의료용전기제품 기술심의회(A,C,P,X)",IEC/TC 62/SC D(의료용전자기기),...,정상오,043-719-5660,"[{'대응국제표준': 'IEC 60601-2-6 Ed. 2.1', '부합화수준': ...",0 

In [40]:
import json
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import collections
collections.Callable = collections.abc.Callable
import pymysql
import pandas as pd
import datetime


# json file로 변환하는 함수
def transform_to_json(dic, file_name):
    with open(file_name, 'w', encoding = 'UTF8') as f:
        json.dump(dic, f, ensure_ascii=False, indent = '\t')
        
        
# text 데이터 정제 함수        
def clean_data(string):
    regexed_str = re.sub("[\n]+", " ", string)
    regexed_str = re.sub("[\t]+", " ", string)
    return_string = regexed_str.replace("\n", " ")
    return return_string.strip()


# 테이블 형태 데이터 처리 함수
def small_table_parser(contents_node, header_node):
    total_lst = []
    for contents_idx in range(0, len(contents_node), len(header_node.find_all("th"))):
        sub_dict = {}
        for th in range(len(header_node.find_all("th"))):
            sub_dict[header_node.find_all("th")[th].text] = clean_data(contents_node[contents_idx + th].text)
        total_lst.append(sub_dict)
    return total_lst


# 표준 데이터 크롤러
def std_crawler(whole_dict, lst_page_soup, driver, std_idx):
    page_list_data = lst_page_soup.find_all("tbody")[1]
    row_lst = page_list_data.find_all("tr")
    
    driver.implicitly_wait(100)
    
    for standard in range(std_idx, len(row_lst) + 1):
        
        # data 개수 체크
        num = int(lst_page_soup.select("#tabs-container > div.table.list > table > tbody > tr:nth-of-type({}) > td:nth-of-type(1)".format(standard))[0].text)
        
        driver.implicitly_wait(100)
        
        #crawling 작업
        driver.find_element(By.XPATH, '//*[@id="tabs-container"]/div[2]/table/tbody/tr[{}]/td[2]/a'.format(standard)).click()
        
        std = {}
        
        result_html = driver.page_source
        soup = BeautifulSoup(result_html, 'html.parser')
        
        # 테이블 row 개수
        table_data = soup.find("div", {'class':'table view'})
        table_data = table_data.find('tbody').children

        table_lst = []
        for child in table_data:
            table_lst.append(child)
        for i in table_lst:
            if i == "\n":
                table_lst.remove(i)
        
                                
        for row in range(1, len(table_lst) + 1):
            selector = "#contents > div > div.content_inner > div.table.view > table > tbody > "
            title = soup.select(selector + "tr:nth-of-type({}) > th".format(row))[0].text
            
            # 인용표준, 기술기준 데이터 / index text를 비교하는 if 문으로 바꾸기 / 가독성 고려
            if title == "인용표준" or title == "기술기준":
                title = soup.select(selector + "tr:nth-of-type({}) > th".format(row))[0].text
                content = soup.select(selector + "tr:nth-of-type({}) > td".format(row))[0].text
                
                title_2 = soup.select(selector + "tr:nth-of-type({}) > th:nth-of-type(2)".format(row))[0].text
                content_2 = soup.select(selector + "tr:nth-of-type({}) > td:nth-of-type(2)".format(row))[0].text
                
                
                content = clean_data(content)
                content_2 = clean_data(content_2)
                
                std[title] = content
                std[title_2] = content_2
                
            #테이블 데이터 처리 / 유형별로 파악해서 if 문 바꾸기 / 아예 클래스를 만들어서 기존 항목에 추가 제거의 경우에 처리할 경우 / 테이블 파싱하는 함수 만들기 (colspan 예외 처리도)
            elif title == "국제표준 부합화" or title == "표준이력사항" or title == "인증심사기준":
                std[title] = []
                
                # 국제표준 부합화
                if title == "국제표준 부합화":
                    sub_dict = {}
                    data = soup.find("div", {'class':"table list gray"})
                    contents = data.find_all("td")
                    
                    if "colspan" in contents[0].attrs:
                        for j in range(int(contents[0].attrs["colspan"])):
                            sub_dict[data.find_all("th")[j].text] = contents[0].text
                        std[title].append(sub_dict)
                        contents = contents[1:]
                        
                    table_data_lst = small_table_parser(contents, data)
                    for tb_data in table_data_lst:
                        std[title].append(tb_data)
                    
                #표준이력사항
                elif title == "표준이력사항":
                    data = soup.find_all("div", {'class':"table list gray"})[1]
                    contents = data.find_all("td")
                    
                    table_data_lst = small_table_parser(contents, data)
                    std[title] = table_data_lst
                        
                #인증심사기준
                else:
                    sub_dict = {}
                    data = soup.find_all("div", {'class':"table list gray"})[2]
                    contents = data.find_all("td")
                    if len(contents) == 1:
                        for i in range(int(contents[0].attrs["colspan"])):
                            sub_dict[data.find_all("th")[i].text] = contents[0].text
                        std[title].append(sub_dict)
                    
                    else:
                        table_data_lst = small_table_parser(contents, data)
                        std[title] = table_data_lst

            # 나머지 데이터
            else:
                content = soup.select(selector + "tr:nth-of-type({}) > td".format(row))[0].text
                content = clean_data(content)
                std[title] = content
                
        now = time.localtime()
        now = "%04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
        std['crawled_time'] = now
        whole_dict['stds'].append(std)
        
        driver.back()
        driver.refresh()
    
        driver.implicitly_wait(100)
    
    std_idx = 1    
    is_success = True
    return whole_dict, is_success, std_idx


# 페이지 넘기는 크롤러
def page_crawler(start_page, chrome_path, url, std_idx):
    driver = webdriver.Chrome(executable_path=chrome_path)
    driver.get(url)
    
    driver.find_element(By.CLASS_NAME, "last").click()
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    last_page = soup.find_all("a", {"class": "on"})[1].text
    driver.find_element(By.CLASS_NAME, "first").click()
    
    std_dict = {}
    std_dict['stds'] = []
    
    start_page = int(start_page)
    if start_page % 10 == 0:
        num_next = start_page // 10 - 1
    else:
        num_next = start_page // 10

    for i in range(num_next):
        driver.find_element(By.CLASS_NAME, 'next').click()
        
    page_source = driver.page_source
    lst_page_soup = BeautifulSoup(page_source, 'html.parser')

    # 페이지 버튼 갯수
    page_button_data = lst_page_soup.find("div", {'class':"page"})
    li_lst = page_button_data.find_all("li")
    li_lst = [button.text for button in li_lst]
    start = li_lst.index(str(start_page))
    
    is_success = False
    page_num = 0
    
    while True:
        for page in range(start + 1, len(li_lst) - 1):
            driver.find_element(By.XPATH, '//*[@id="tabs-container"]/div[3]/div/div/ul/li[{}]/a'.format(page)).click()
            page_num = driver.find_element(By.XPATH, '//*[@id="tabs-container"]/div[3]/div/div/ul/li[{}]/a'.format(page)).text
            page_source = driver.page_source
            lst_page_soup = BeautifulSoup(page_source, 'html.parser')
            driver.implicitly_wait(100)
            
            std_dict, is_success, std_idx = std_crawler(std_dict, lst_page_soup, driver, std_idx)
            
            
            if page_num == last_page:
                is_success = True
                return std_dict, int(page_num), is_success, std_idx
            
            elif is_success == False:
                return std_dict, int(page_num), is_success, std_idx
                
            
        # 다시 1페이지부터
        start = 2
        
        driver.find_element(By.CLASS_NAME, 'next').click()
         
    

# json 파일 dataframe으로 읽는 함수
def json_reader(filename):
    with open (filename, "r", encoding = 'UTF8') as f:
        data = json.load(f)
    
    df = pd.json_normalize(data['stds'])
    return df


# DB에 테이블이 생성되어 있다면 테이블에 데이터 넣는 함수
def db_process(filename, host, password, schema_name, table_name):
    try:
        db = pymysql.connect(host = host, port = 3306, user = 'root', password = password,
                        db = schema_name, charset = 'utf8')   # charset: 인코딩 설정
    except: 
        print("Can not connent to DB server. Please check the connection")
        return
    
    cursor = db.cursor()

    sql = 'INSERT INTO ' + table_name + ' (doc_num, doc_name_ko, doc_name_en, publish_date, final_date, json_data, crawled_time, tag) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'

    output_df = json_reader(filename)
    output_df = output_df.reset_index()

    with open (filename, "r", encoding = 'UTF8') as f:
        data = json.load(f)
        
    lst = data['stds']
            
    for i in range(len(output_df)):
        json_data = json.dumps(lst[i])
        try:
            cursor.execute(sql, (output_df.loc[i]["표준번호"], output_df.loc[i]["표준명(한글)"], output_df.loc[i]["표준명(영문)"], output_df.loc[i]["제정일"], output_df.loc[i]["최종개정확인일"], json_data, output_df.loc[i]["crawled_time"], filename[7:15]))
        except:
            if len(output_df.loc[i]["제정일"]) == 0:
                cursor.execute(sql, (output_df.loc[i]["표준번호"], output_df.loc[i]["표준명(한글)"], output_df.loc[i]["표준명(영문)"], None, output_df.loc[i]["최종개정확인일"], json_data, output_df.loc[i]["crawled_time"], filename[7:15]))
            else:
                print("There is a problem while inserting data number : ", i)
    
    try:
        db.commit()
    except Exception as ex:
        print(ex)
        print("db commite error")

    db.close()
    print("Done with inserting data to DB")
    return


# main 함수 = crawling --> json data --> DB 연동 및 데이터 inserting
def main(last_page_no, max_retry_num, chrome_path, url, host, password, schema_name, table_name):
    data_list = {}
    data_list['stds'] = []
    current_retry_count = 0
    std_idx = 1
    now = datetime.datetime.now()
    output_name = "output_{}.json".format(now.strftime('%Y%m%d'))
    
    while True:
        if current_retry_count >= max_retry_num:
            print("number of retry exceeds maximum retry number")
            print("last page of crawling is ", last_page_no)
            transform_to_json(data_list, output_name)
            break
            # return data_list, is_success, current_retry_count
        
        current_data_list, end_page_no, is_success, std_idx = page_crawler(last_page_no, chrome_path, url, std_idx)
        
        # merge current_data_list to data_list 
        for i in current_data_list['stds']:
            if i not in data_list['stds']:
                data_list['stds'].append(i)
                
        # if it couldn't reach the last page // 8901 and 10001에서 클릭 오류 해결하기 위해 driver 새로 한개 만들어줌
        if is_success == False:
            current_retry_count = current_retry_count + 1
            transform_to_json(data_list, output_name)
            last_page_no = end_page_no
            
            # print("Trying to crawl the error page : ", last_page_no)
            
            # driver_2 = webdriver.Chrome(executable_path=chrome_path)
            # driver_2.get(url)

            # for i in range(end_page_no // 10):
            #     driver_2.find_element(By.CLASS_NAME, 'next').click()
                
            # page_source = driver_2.page_source
            # lst_page_soup = BeautifulSoup(page_source, 'html.parser')
            
            # sub_dict = {}
            # sub_dict['stds'] = []

            # not_success_dict, is_sucess, standard = std_crawler(sub_dict, lst_page_soup, driver_2, std_idx)
            
            # for i in not_success_dict['stds']:
            #     data_list['stds'].append(i)
            
            # print("Crawling is finished with the error page : ", last_page_no)

            # last_page_no = last_page_no + 1

            print("Recrawling from the page number : ", last_page_no)
        
        # if it reaches the last page
        elif is_success:
            print("Done with crawling")
            transform_to_json(data_list, output_name)
            break
            # return data_list, is_success, current_retry_count
            
    return db_process(output_name, host, password, schema_name, table_name)
        
        
url = "https://standard.go.kr/KSCI/standardIntro/getStandardSearchList.do?menuId=919&topMenuId=502"
main(891, 10, r"C:\Users\gihok\chatbot\chromedriver.exe", url, "192.168.0.124", "linuxer", "std_crawled_data", "std_data_check")

C:\Users\gihok\AppData\Local\Temp\ipykernel_11200\1076415428.py:157: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_path)


ElementClickInterceptedException: Message: element click intercepted: Element <a href="javascript:fnGoStdView('KSCIEC60601-2-8','KSCIEC60601-2-8','05');">...</a> is not clickable at point (439, 29). Other element would receive the click: <a href="javascript:menu(4)">...</a>
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x01006463+2188387]
	Ordinal0 [0x00F9E461+1762401]
	Ordinal0 [0x00EB3D78+802168]
	Ordinal0 [0x00EE7F9B+1015707]
	Ordinal0 [0x00EE5F68+1007464]
	Ordinal0 [0x00EE3C6B+998507]
	Ordinal0 [0x00EE29D9+993753]
	Ordinal0 [0x00ED8613+951827]
	Ordinal0 [0x00EFC7DC+1099740]
	Ordinal0 [0x00ED7FF4+950260]
	Ordinal0 [0x00EFC9F4+1100276]
	Ordinal0 [0x00F0CC22+1166370]
	Ordinal0 [0x00EFC5F6+1099254]
	Ordinal0 [0x00ED6BE0+945120]
	Ordinal0 [0x00ED7AD6+948950]
	GetHandleVerifier [0x012A71F2+2712546]
	GetHandleVerifier [0x0129886D+2652765]
	GetHandleVerifier [0x0109002A+520730]
	GetHandleVerifier [0x0108EE06+516086]
	Ordinal0 [0x00FA468B+1787531]
	Ordinal0 [0x00

In [49]:
page = open("page_source891.html", 'rt', encoding = 'utf8').read()
soup = BeautifulSoup(page, 'html.parser')
soup

<html><head>
<meta charset="utf-8">
<meta content="IE=edge" http-equiv="X-UA-Compatible">
<title>표준 검색 &gt; 국가표준 | e나라 표준인증</title>
<script type="text/javascript"> 
//history.pushState(null, null, location.href);
//window.onpopstate = function(event) {
//	history.go(1);
//};
</script>
<meta content="IE=edge" http-equiv="X-UA-Compatible">
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<link href="/KSCI/portal/user/css/contents.css" rel="stylesheet" type="text/css">
<link href="/KSCI/portal/user/css/jquery-ui-1.10.2.custom.css" rel="stylesheet" type="text/css">
<link href="/KSCI/portal/user/css/showLoading.css" rel="stylesheet" type="text/css">
<link href="/KSCI/portal/user/css/chosen.css" rel="stylesheet" type="text/css">
<link href="/KSCI/portal/user/tsr3/css/style.css" rel="stylesheet" type="text/css">
<script charset="utf-8" language="javascript" src="/KSCI/portal/user/tsr3/js/jquery-1.11.1.min.js" type="text/javascript"></script>
<script charset="utf-8" language

In [57]:
page_1 = open("page_source1.html", 'rt', encoding = 'utf8').read()
soup_1 = BeautifulSoup(page, 'html.parser')
soup_1

<html><head>
<meta charset="utf-8">
<meta content="IE=edge" http-equiv="X-UA-Compatible">
<title>표준 검색 &gt; 국가표준 | e나라 표준인증</title>
<script type="text/javascript"> 
//history.pushState(null, null, location.href);
//window.onpopstate = function(event) {
//	history.go(1);
//};
</script>
<meta content="IE=edge" http-equiv="X-UA-Compatible">
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<link href="/KSCI/portal/user/css/contents.css" rel="stylesheet" type="text/css">
<link href="/KSCI/portal/user/css/jquery-ui-1.10.2.custom.css" rel="stylesheet" type="text/css">
<link href="/KSCI/portal/user/css/showLoading.css" rel="stylesheet" type="text/css">
<link href="/KSCI/portal/user/css/chosen.css" rel="stylesheet" type="text/css">
<link href="/KSCI/portal/user/tsr3/css/style.css" rel="stylesheet" type="text/css">
<script charset="utf-8" language="javascript" src="/KSCI/portal/user/tsr3/js/jquery-1.11.1.min.js" type="text/javascript"></script>
<script charset="utf-8" language

In [58]:
soup_1.select("#tabs-container > div.table.list > table > tbody > tr:nth-of-type(1) > td:nth-of-type(2) > a")

[<a href="javascript:fnGoStdView('KSCIEC60601-2-8','KSCIEC60601-2-8','05');">KS C IEC60601-2-8</a>]

In [54]:
soup.select("#tabs-container > div.table.list > table > tbody > tr:nth-of-type(1) > td:nth-of-type(2) > a")

[<a href="javascript:fnGoStdView('KSCIEC60601-2-8','KSCIEC60601-2-8','05');">KS C IEC60601-2-8</a>]